In [8]:
import pandas as pd

In [9]:
data = pd.read_csv('msibSentiment.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   conversation_id_str      1396 non-null   object 
 1   created_at               1396 non-null   object 
 2   favorite_count           1395 non-null   float64
 3   full_text                1396 non-null   object 
 4   id_str                   1395 non-null   object 
 5   image_url                581 non-null    object 
 6   in_reply_to_screen_name  413 non-null    object 
 7   lang                     1395 non-null   object 
 8   location                 959 non-null    object 
 9   quote_count              1395 non-null   object 
 10  reply_count              1395 non-null   float64
 11  retweet_count            1395 non-null   object 
 12  tweet_url                1394 non-null   object 
 13  user_id_str              1394 non-null   float64
 14  username                

In [10]:
data.drop(['conversation_id_str', 'created_at', 'favorite_count', 'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 'retweet_count', 'tweet_url', 'user_id_str', 'username', 'text', 'full_text'], axis=1, inplace=True)


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text_clean  1393 non-null   object 
 1   polarity    1396 non-null   float64
 2   sentiment   1396 non-null   object 
dtypes: float64(1), object(2)
memory usage: 32.8+ KB


In [12]:
data.sample(n=10)

,text_clean,polarity,sentiment
82,sosialisasi program magang studi independen be...,-0.4,Negative
1352,daftar program magang studi independen bersert...,0.0,Neutral
379,open menerima jasa bikin cv at friendli cv kre...,0.0,Neutral
1358,repost from atrbpn program magang studi indepe...,0.0,Neutral
511,bentar msib magang studi independen bersertifi...,0.0,Neutral
142,galau bgt msib bgt huhu magang jurusna gitu me...,0.0,Neutral
1321,kali sehubungan terlaksananya program magang s...,0.0,Neutral
974,ntb ntt bund karna jg udh samp lombok tuh expl...,0.0,Neutral
528,mahasiswa polinema mengikuti program magang st...,0.0,Neutral
733,webinar kinobi x msib kampu merdeka kesempatan...,0.0,Neutral


In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# Definisikan fungsi untuk mengolah teks
def preprocess_text(text):
    # Konversi ke huruf kecil
    text = str(text).lower()

    # Hapus karakter khusus dan tanda baca
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenisasi teks
    tokens = word_tokenize(text)

    # Hapus kata stop
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Aplikasikan pengolah teks ke data teks Anda
data['text_clean'] = data['text_clean'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text_clean'])

# Convert text to sequences of integers
X = tokenizer.texts_to_sequences(data['text_clean'])

# Pad sequences to ensure uniform length
X_padded = pad_sequences(X)

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_padded, data['sentiment'], test_size=0.2, random_state=42)

# Encode class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Inisialisasi model LSTM dengan Keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Latih model
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, verbose=1)

# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print("Akurasi:", accuracy)

# Prediksi menggunakan model yang sudah dilatih
predictions = (model.predict(X_test) > 0.5).astype("int32")

print("\nLaporan Klasifikasi:")
print(classification_report(y_test_encoded, predictions))

Epoch 1/10
35/35 [==============================] - 2s 20ms/step - loss: 0.1918 - accuracy: 0.9194
Epoch 2/10
35/35 [==============================] - 1s 22ms/step - loss: -0.0689 - accuracy: 0.9409
Epoch 3/10
35/35 [==============================] - 1s 29ms/step - loss: -0.0820 - accuracy: 0.9409
Epoch 4/10
35/35 [==============================] - 1s 34ms/step - loss: -0.1047 - accuracy: 0.9409
Epoch 5/10
35/35 [==============================] - 1s 25ms/step - loss: -0.1286 - accuracy: 0.9409
Epoch 6/10
35/35 [==============================] - 1s 21ms/step - loss: -0.1868 - accuracy: 0.9409
Epoch 7/10
35/35 [==============================] - 1s 21ms/step - loss: -0.3238 - accuracy: 0.9409
Epoch 8/10
35/35 [==============================] - 1s 20ms/step - loss: -0.4724 - accuracy: 0.9516
Epoch 9/10
35/35 [==============================] - 1s 21ms/step - loss: -0.5672 - accuracy: 0.9606
Epoch 10/10
9/9 [==============================] - 0s 6ms/step - loss: -0.1889 - accuracy: 0.9607
Aku

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
